In [1]:
import torch
import numpy as np
import pandas as pd
from helper import *
import transformers
from transformers import AutoTokenizer, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('/content/UpdatedResumeDataSet.csv')

df

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."
...,...,...
957,Testing,Computer Skills: â¢ Proficient in MS office (...
958,Testing,â Willingness to accept the challenges. â ...
959,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne..."
960,Testing,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...


In [3]:
df['Category'].unique()

array(['Data Science', 'HR', 'Advocate', 'Arts', 'Web Designing',
       'Mechanical Engineer', 'Sales', 'Health and fitness',
       'Civil Engineer', 'Java Developer', 'Business Analyst',
       'SAP Developer', 'Automation Testing', 'Electrical Engineering',
       'Operations Manager', 'Python Developer', 'DevOps Engineer',
       'Network Security Engineer', 'PMO', 'Database', 'Hadoop',
       'ETL Developer', 'DotNet Developer', 'Blockchain', 'Testing'],
      dtype=object)

In [4]:
label2id = {}

for index,job in enumerate(df['Category'].unique()):
    label2id[job] = index

id2label  = {v:u for v,u in enumerate(label2id)}


df['Category'] = df['Category'].map(lambda x:label2id[x])

In [6]:
import re
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


def preprocess_text(text):
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text(separator=' ')

    text = text.lower()

    # Lowercase the text
    contractions = {
        r"\b(can't)\b": "can not",
        r"\b(ain't)\b": "am not",
        r"\b(aren't)\b": "are not",
        r"\b(couldn't)\b": "could not",
        r"\b(didn't)\b": "did not",
        r"\b(doesn't)\b": "does not",
        r"\b(don't)\b": "do not",
        r"\b(hadn't)\b": "had not",
        r"\b(hasn't)\b": "has not",
        r"\b(haven't)\b": "have not",
        r"\b(isn't)\b": "is not",
        r"\b(there's)\b": "there is",
        r"\b(wasn't)\b": "was not",
        r"\b(why's)\b": "why is",
        r"\b(won't)\b": "will not",
        r"\b(wouldn't)\b": "would not",
    }

    # Replace contractions with their expanded forms
    for pattern, replacement in contractions.items():
        text = re.sub(pattern, replacement, text)

    text = text.replace("'s",' is')
    text = text.replace("'m",' am')
    text = text.replace("'re",' are')
    text = text.replace("'ve",' have')
    text = text.replace("'ll",' will')
    text = text.replace("'d",' would')
    text = text.replace("n't",' not')
    text = text.replace("'t",' not')

    # Tokenization
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Remove punctuations
    tokens = [token for token in tokens if token.isalpha() or token.isalnum()]

    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    processed_text = ' '.join(tokens)
    return processed_text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [7]:
df['Resume'] = df['Resume'].apply(preprocess_text)

df

,Category,Resume
0,0,skill program languag python panda numpi scipi...
1,0,educ detail may 2013 may 2017 data scientist d...
2,0,area interest deep learn control system design...
3,0,skill r python sap hana tableau sap hana sql s...
4,0,educ detail mca ymcaust faridabad haryana data...
...,...,...
957,24,comput skill profici m offic word basic excel ...
958,24,willing accept challeng posit think good learn...
959,24,person skill quick learner eager learn new thi...
960,24,comput skill softwar knowledg point m offic c ...


In [9]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00


In [10]:
from datasets import Dataset, DatasetDict, ClassLabel

dataset = Dataset.from_pandas(df)

split_number = int(0.8*len(df))
train_dataset = dataset.shuffle(seed=42).select(range(split_number))
test_dataset = dataset.select(range(split_number, len(df)))

In [11]:
dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

dataset

DatasetDict({
    train: Dataset({
        features: ['Category', 'Resume'],
        num_rows: 769
    })
    test: Dataset({
        features: ['Category', 'Resume'],
        num_rows: 193
    })
})

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["Resume"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/769 [00:00<?, ? examples/s]

Map:   0%|          | 0/193 [00:00<?, ? examples/s]

In [13]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['Category', 'Resume', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 769
    })
    test: Dataset({
        features: ['Category', 'Resume', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 193
    })
})

In [14]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=25)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
tokenized_datasets = tokenized_datasets.remove_columns(["Resume"])
tokenized_datasets = tokenized_datasets.rename_column("Category", "labels")
tokenized_datasets.set_format("torch")


In [16]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 769
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 193
    })
})

In [17]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_datasets['train'], shuffle=True, batch_size=8)
eval_dataloader = DataLoader(tokenized_datasets['test'], batch_size=8)

In [18]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [19]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [20]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [21]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/291 [00:00<?, ?it/s]

In [23]:
!pip3 install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00


In [31]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:

    batch = {k: v.to(device) for k, v in batch.items()}


    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 1.0}

In [32]:
model.save_pretrained("./pretrained_model")